# Notebook Overview

## Motivation 

Tabular data reasoning in the financial domain. Multiple reasoning steps may be needed to identify the right answer. For example, a financial analyst working with financial statements may be interested in computing various financial ratios with natural language queries.

Some examples may include:
- ROE (Return on Equity) = Net Income / Shareholder's Equity
- Net Profit Margin = Net Income / Revenue
- Asset Turnover = Revenue / Average Total Assets
- Financial Leverage = Average Total Assets / Shareholder's Equity

Having an Agent which is able to correctly compute these and other ratios would be a great help for any analyst in the field of Finance.

## Objective

In this notebook we explore how to setup a [Cohere Agent](https://docs.cohere.com/docs/multi-step-tool-use) to answer questions over tables in Apple's SEC10K 2020 form. This notebook uses Cohere's native API with langchain python tool. 

## Table of Contents

- [Setup](#setup)
- [Define Python Tool](#define_python_tool)
- [Create Cohere Agent](#create_cohere_agent)
- [QnA over Single Table](#qna_over_single_table)
- [QnA over Multiple Tables](#qna_over_multiple_tables)
- [Error Resilience](#error_resilience)
- [Add Viewing Tool](#add_viewing_tool)


<a id="setup"></a>

# Setup

In [ ]:
####################################################################################################
#
# Uncomment if you need to install the following packages
#
####################################################################################################

# !pip install --quiet langchain langchain_experimental cohere --upgrade

In [ ]:
#!pip install --quiet typing langchain langchain_experimental cohere httpx --upgrade

In [ ]:
import os
from typing import List

import httpx
import pydantic

import cohere
import langchain
import langchain_core
import langchain_experimental
import pandas as pd
from langchain.agents import Tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.utilities import PythonREPL

In [ ]:
# versions
print('cohere version:', cohere.__version__)
print('langchain version:', langchain.__version__)
print('langchain_core version:', langchain_core.__version__)
print('langchain_experimental version:', langchain_experimental.__version__)

### API Key 
This code is presenly desiged to run on Google Colab. If you decide to run it elsewhere, you need to 
adapt the code to pull the API Key accordingly.

In [ ]:
COHERE_API_KEY = os.environ["COHERE_API_KEY"]
COHERE_MODEL = 'command-r-plus'
co = cohere.Client(api_key=COHERE_API_KEY)

### Data Loading 
These data files have been processed to be used by this notebook. The original forms require 
additional text processing to be extract these data.

In [ ]:
income_statement = pd.read_csv('income_statement.csv')
balance_sheet = pd.read_csv('balance_sheet.csv')

In [ ]:
income_statement.head(2)

In [ ]:
balance_sheet.head(2)

<a id="define_python_tool"></a>

# Define Python Tool 

Here we define the python tool using langchain's PythonREPL. We also define `functions_map` that will later be used by the Cohere Agent to correctly map function name to the actual function. Lastly, we define the tools that will be passed in the Cohere API. 


In [ ]:
python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
python_tool.name = "python_interpreter"

class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
python_tool.args_schema = ToolInput

def run_python_code(code: str) -> dict:
    """
    Function to run given python code
    """
    input_code = ToolInput(code=code)
    return {'python_answer': python_tool.func(input_code.code)}

functions_map = {
    "run_python_code": run_python_code,
}

tools = [
    {
        "name": "run_python_code",
        "description": "given a python code, runs it",
        "parameter_definitions": {
            "code": {
                "description": "executable python code",
                "type": "str",
                "required": True
            }
        }
    },]

<a id="create_cohere_agent"></a>

# Create Cohere Agent 

As [Vanilla_Multi_Step_Tool_Use.ipynb](https://github.com/cohere-ai/notebooks/blob/fbf6c8dad47d7557314e9248a267175c7a6908d8/notebooks/Vanilla_Multi_Step_Tool_Use.ipynb) shows, you have a lot of flexiblity on how you can customize and interact with the cohere agent. Here I am creating a wrapper so that it automatically determines when to stop calling the tools and output final answer. It will run maximum of 15 steps. 

In [ ]:
def cohere_agent(
    message: str,
    preamble: str,
    tools: List[dict],
    force_single_step=False,
    verbose: bool = False,
) -> str:
    """
    Function to handle multi-step tool use api.

    Args:
        message (str): The message to send to the Cohere AI model.
        preamble (str): The preamble or context for the conversation.
        tools (list of dict): List of tools to use in the conversation.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        str: The final response from the call.
    """

    counter = 1

    response = co.chat(
        model=COHERE_MODEL,
        message=message,
        preamble=preamble,
        tools=tools,
        force_single_step=force_single_step,
    )

    if verbose:
        print(f"\nrunning 0th step.")
        print(response.text)

    while response.tool_calls:
        tool_results = []

        if verbose:
            print(f"\nrunning {counter}th step.")

        for tool_call in response.tool_calls:
            output = functions_map[tool_call.name](**tool_call.parameters)
            outputs = [output]
            tool_results.append({"call": tool_call, "outputs": outputs})

            if verbose:
                print(
                    f"= running tool {tool_call.name}, with parameters: {tool_call.parameters}"
                )
                print(f"== tool results: {outputs}")

        response = co.chat(
            model=COHERE_MODEL,
            message="",
            chat_history=response.chat_history,
            preamble=preamble,
            tools=tools,
            force_single_step=force_single_step,
            tool_results=tool_results,
        )

        if verbose:
            print(response.text)

            counter += 1

    return response.text


# test
output = cohere_agent("can you use python to answer 1 + 1", None, tools, verbose=True)

<a id="qna_over_single_table"></a>

# QnA over Single Table 

In the example below, we show how the python tool can be used to load a dataframe and extract information from it. To do this successfully we need to:

1) pass the file name to the preamble so the model knows how to load the dataframe
2) pass a preview of the dataframe in the preamble so the model knows which columns/rows to query

We will ask the following questions given income statement data. 
1. what is the highest value of cost of goods and service?
2. what is the largest gross profit margin?
3. what is the minimum ratio of operating income loss divided by non operating income expense? 

In [ ]:
question_dict ={
    'q1': ['what is the highest value of cost of goods and service?',169559000000],
    'q2': ['what is the largest gross profit margin?',0.3836194330595236],
    'q3': ['what is the minimum ratio of operating income loss divided by non operating income expense?',35.360599]
}

In [ ]:
preamble = """
You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
Here is a preview of the dataframe:
{head_df}
""".format(head_df=income_statement.head(3).to_markdown())

print(preamble)

In [ ]:
for qsn,val in question_dict.items():
    print(f'question:{qsn}')
    question = val[0]
    answer = val[1]
    output = cohere_agent(question, preamble, tools, verbose=True)
    print(f'GT Answer:{val[1]}')
    print('-'*50)

<a id="qna_over_multiple_tables"></a>

# QnA over Multiple Tables 

We now make the task for the Agent more complicated, by asking it a question the answer for which can be computed only by retrieving relevant information from multiple tables: 

- Q: What is the ratio of the largest stockholders equity to the smallest revenue?

As you will see below, this question can be obtained only by accessing both the balance sheet and the income statement. 



In [ ]:
question_dict ={
    'q1': ['what is the ratio of the largest stockholders equity to the smallest revenue'],
}

In [ ]:
# get the largest stockholders equity
x = balance_sheet['StockholdersEquity'].astype(float).max()
print(f"The largest stockholders equity value is: {x}")

# get the smallest revenue
y = income_statement['RevenueFromContractWithCustomerExcludingAssessedTax'].astype(float).min()
print(f"The smallest revenue value is: {y}")

# compute the ratio
ratio = x/y
print(f"Their ratio is: {ratio}")

In [ ]:
preamble = """
You are an expert who answers the user's question in complete sentences. You are working with two pandas dataframe in Python. Ensure your output is a string.

Here is a preview of the `income_statement.csv` dataframe:
{table_1}

Here is a preview of the `balance_sheet.csv` dataframe:
{table_2}
""".format(table_1=income_statement.head(3).to_markdown(),table_2=balance_sheet.head(3).to_markdown())


print(preamble)

In [ ]:
for qsn,val in question_dict.items():
    print(f'question:{qsn}')
    question = val[0]
    output = cohere_agent(question, preamble, tools, verbose=True)

<a id="error_resilience"></a>

# Error Resilience

In the previous example over single table, the model successfully answered the questions. However, the model may not always have access to the preview of the data. Wwhen we remove the preview from the preamble, the model runs into an error and will not produce the answer. We will solve this problem in two different ways: 

1. Asking the model to keep trying until it fixes the issue. 
2. Giving the model another tool to view the data and telling it to preview the data before writing code. 

The second method allows the model to obtain the answer with fewer steps. 


In [ ]:
preamble = """
You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
"""

question1 = "what is the highest value of cost of goods and service?"

output = cohere_agent(question1, preamble, tools, verbose=True)

As you see above, the model failed to execute because it assumed certain column names but they turned out to be wrong. One simple fix is to tell the model to continue to solve the problem in the face of error. 

In [ ]:
preamble = """
You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
If you run into error, keep trying until you fix it. You may need to view the data to understand the error.
"""

question1 = "what is the highest value of cost of goods and service?"

output = cohere_agent(question1, preamble, tools, verbose=True)

<a id="add_viewing_tool"></a>

### Add Viewing Tool 

What if we directly give the model the ability to view the data as a tool so that it can explicitly use it instead of indirectly figuring it out? 

In [ ]:
def view_csv_data(path: str) -> dict:
    """
    Function to view the head, tail and shape of a given csv file.
    """
    df = pd.read_csv(path)

    return {
        "head": df.head().to_string(),
        "tail": df.tail().to_string(),
        "shape": str(df.shape),
    }

functions_map = {
    "run_python_code": run_python_code,
    "view_csv_data": view_csv_data
}

tools = [
    {
        "name": "run_python_code",
        "description": "given a python code, runs it",
        "parameter_definitions": {
            "code": {
                "description": "executable python code",
                "type": "str",
                "required": True
            }
        }
    },
    {
        "name": "view_csv_data",
        "description": "give path to csv data and get head, tail and shape of the data",
        "parameter_definitions": {
            "path": {
                "description": "path to csv",
                "type": "str",
                "required": True
            }
        }
    },
]

In [ ]:
preamble = """
You are an expert who answers the user's question. You are working with a pandas dataframe in Python. The name of the dataframe is `income_statement.csv`.
Always view the data first to write flawless code.
"""

question1 = "what is the highest value of cost of goods and service?"

output = cohere_agent(question1, preamble, tools, verbose=True)

By being prescriptive, we were able to cut down a step and get to the answer faster. 